##### **전체 로드맵**

In [ ]:
# 지능정보SW아카데미 -> SW(프로그래밍) + 지능정보(AI)
#                   -> 코딩        -> 모델링(데이터가 필수)
#                                 -> 빅데이터 (많은, 다양한, 신속한...)

In [1]:
# INISW => INI(intelligence and informatics)
# 데이터수집 - 데이터관리 - 분석(모델링) - 응용(시각화/SW) => 우리는 이 과정을 Python을 통해서 함

# 1.        (R)DB(MS) - SQLite : 서비스(비즈니스 로직) != 코드산출물(*) 따라서
#                       ORM (SQLAlchemy - Core / ORM ) 사용
#
#     -> 이 때 다루는 데이터는 Structured Data
#
# 2. 수집: Crawler(Crawling+Scrapping)  IR(정보검색. 텍스트 마이닝에 속함)
#     -> Unstructured / Semi(Documnet) Data
#   (Image+Audio+Video...) / (SGML/XML/HTML/JSON...)
#                              -> 형식이 있으니 Parsing을 해야 함
#                                 그 방법은 RE(정규식), DOM, CSS Selctor, Xpath, ...
#
# 3. 분석: 텍스트(한글) 전처리 + ML(머신러닝, Classification+Clustering/Topic)
#
# 4. Web, App => Server(C/S, 서버 클라이언트)  + UI/UX
#   Rest(ful) API

##### **ORM**

In [5]:
import sqlalchemy

In [6]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.schema import MetaData, Table, Column, ForeignKey
from sqlalchemy.types import Integer, Text

In [8]:
# ORM 으로 데이터 관리하기 -> Object / Data 간에 Mapping 해준다는 것
# Data Mapping 은 engine 영역에서 일어남.
# 우리가 건드릴 것은 Object 즉 Class
base = sqlalchemy.orm.declarative_base()
# 선언적 class 정의를 위한 base 만들기


In [9]:
# base를 상속하는 class 선언
class TableA(base):
    __tablename__ = 'T_A'
    
    PK = Column('pk', Integer, primary_key=True)
    NAME = Column('name', Text, nullable=False) # Not Null
    

In [10]:
TableA.__table__
# 이게 어제 우리가 코어 단에서 만들었던 테이블 객체

Table('T_A', MetaData(), Column('pk', Integer(), table=<T_A>, primary_key=True, nullable=False), Column('name', Text(), table=<T_A>, nullable=False), schema=None)

In [11]:
TableA.__tablename__

'T_A'

In [ ]:
# 어제는 메타데이터에 테이블 객체를 만들어주고 엔진에 바인드 시켜서 수행해줌.

In [12]:
a = TableA(NAME='아무나') # 인스턴스

In [127]:
a = TableA(NAME=['블라블라', '오우오우'])

In [13]:
# 지금은 순수하게 DB랑 아무 상관 없음
a.PK, a.NAME
# 지금까지는 메모리상에 클래스와 인스턴스로만 존재하는 값

(None, '아무나')

In [14]:
engine = create_engine('sqlite:///:memory:', echo=True)

In [18]:
# 엔진을 바인드 해주는 애가 필요함 -> 어제는 메타데이터에 바인드 시킴
base.metadata.tables

FacadeDict({'T_A': Table('T_A', MetaData(), Column('pk', Integer(), table=<T_A>, primary_key=True, nullable=False), Column('name', Text(), table=<T_A>, nullable=False), schema=None)})

In [19]:
# ORM Core
base.metadata.create_all(engine)
# 우리는 메모리에서 작업했지만, 이제야 물리적인 DB에 테이블이 생성됨.

2023-03-08 21:21:41,582 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 21:21:41,582 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_A")
2023-03-08 21:21:41,583 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 21:21:41,584 INFO sqlalchemy.engine.Engine COMMIT


In [20]:
a.PK

In [ ]:
# Session
# 실제 데이터베이스와 오브젝트 간에 왔다갔다 데이터가 매핑되어 있는 것들을 관리해주는 애
# 세션을 통해서 객체를 실행하고 변화시키면 세션이 들고 있다가 값에 변화가 생겨 데이터베이스와 싱크가
# 안맞음을 탐지해줌.
# sessionmaker 로부터 세션을 받고 세션의 인스턴스를 만드는 것부터 시작함. -> 객체와 데이터를 매핑

In [21]:
# ORM Session
from sqlalchemy.orm import sessionmaker

In [22]:
Session = sessionmaker(engine)
session = Session()  # 우리가 작업할 인스턴스

In [18]:
session.is_modified(a), session.dirty
# 전자는 내가 관리하는 '데이터베이스'의 로우 객체에 반영해야할 것이 있는지 -> 변화된 것이 있는지
# 후자는 세션이 관리하는 모든 객체들 중 값의 변화 등이 있을 때 현재 데이터베이스와 싱크가 안 맞으면 알려줌.
# 뒤에 정리해 놓음

(True, IdentitySet([]))

In [23]:
session.add(a)

In [24]:
session.commit()
# 실제 데이터가 들어감

2023-03-08 21:23:09,792 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 21:23:09,794 INFO sqlalchemy.engine.Engine INSERT INTO "T_A" (name) VALUES (?)
2023-03-08 21:23:09,795 INFO sqlalchemy.engine.Engine [generated in 0.00087s] ('아무나',)
2023-03-08 21:23:09,796 INFO sqlalchemy.engine.Engine COMMIT


In [25]:
a.PK
# 아까는 none이었는데, 알아서 나옴. 코어단에서 알아서 처리했기 때문.

2023-03-08 21:23:10,869 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 21:23:10,871 INFO sqlalchemy.engine.Engine SELECT "T_A".pk AS "T_A_pk", "T_A".name AS "T_A_name" 
FROM "T_A" 
WHERE "T_A".pk = ?
2023-03-08 21:23:10,872 INFO sqlalchemy.engine.Engine [generated in 0.00096s] (1,)


1

In [26]:
# 이게 서버단에서 로그인이 되면 사용자의 정보를 불러오는 방식임.
# 세션에 쿼리 주기
a == session.query(TableA).one()
# 데이터가 하나밖에 안 들어가서 같다고 나옴.

2023-03-08 21:24:01,993 INFO sqlalchemy.engine.Engine SELECT "T_A".pk AS "T_A_pk", "T_A".name AS "T_A_name" 
FROM "T_A"
2023-03-08 21:24:01,994 INFO sqlalchemy.engine.Engine [generated in 0.00096s] ()


True

In [27]:
b = session.query(TableA).one()

2023-03-08 21:24:03,839 INFO sqlalchemy.engine.Engine SELECT "T_A".pk AS "T_A_pk", "T_A".name AS "T_A_name" 
FROM "T_A"
2023-03-08 21:24:03,840 INFO sqlalchemy.engine.Engine [cached since 1.847s ago] ()


In [28]:
b.PK # 로그인 회원정보 객체

1

In [29]:
# 정보가 갱신이 일어나면,
b.NAME = '???????'

In [30]:
session.dirty
# session.dirty는 세션이 관리하는 '객체'의 값에 변화가 일어나면 그 객체를 리턴해줌

IdentitySet([<__main__.TableA object at 0x7ff57838b4f0>])

In [31]:
session.commit()
# 실제 데이터베이스의 커밋과는 다르긴 함

2023-03-08 21:24:22,674 INFO sqlalchemy.engine.Engine UPDATE "T_A" SET name=? WHERE "T_A".pk = ?
2023-03-08 21:24:22,675 INFO sqlalchemy.engine.Engine [generated in 0.00088s] ('???????', 1)
2023-03-08 21:24:22,676 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
# 데이터를 가져올 때.
# session.query(TableA).one()은 Select(TableA).fetchone()과 같다고 보면 됨.

In [32]:
session.query(TableA).where(TableA.PK == 1).one().NAME

2023-03-08 21:24:38,462 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 21:24:38,464 INFO sqlalchemy.engine.Engine SELECT "T_A".pk AS "T_A_pk", "T_A".name AS "T_A_name" 
FROM "T_A" 
WHERE "T_A".pk = ?
2023-03-08 21:24:38,465 INFO sqlalchemy.engine.Engine [generated in 0.00080s] (1,)


'???????'

In [33]:
# add_all 은 여러개의 인스턴스들을 다룸
session.add_all([TableA(NAME='2'), TableA(NAME='3'), TableA(NAME='4')])
session.commit() # 커밋을 해야 반영됨

2023-03-08 21:24:51,593 INFO sqlalchemy.engine.Engine INSERT INTO "T_A" (name) VALUES (?), (?), (?) RETURNING pk
2023-03-08 21:24:51,594 INFO sqlalchemy.engine.Engine [generated in 0.00009s (insertmanyvalues)] ('2', '3', '4')
2023-03-08 21:24:51,596 INFO sqlalchemy.engine.Engine COMMIT


In [34]:
inst = session.query(TableA).all()

2023-03-08 21:25:25,415 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 21:25:25,416 INFO sqlalchemy.engine.Engine SELECT "T_A".pk AS "T_A_pk", "T_A".name AS "T_A_name" 
FROM "T_A"
2023-03-08 21:25:25,416 INFO sqlalchemy.engine.Engine [cached since 83.42s ago] ()


In [35]:
a is b, b is inst[0], a is inst[2]

(True, True, False)

In [36]:
inst[0].PK, inst[1].PK, inst[2].PK
# 이렇게 실제 DB를 보지 않고, SQL문이 아니어도 확인할 수 있음

(1, 2, 3)

In [37]:
inst

In [38]:
print(inst)

[<__main__.TableA object at 0x7ff57838b4f0>, <__main__.TableA object at 0x7ff59212d180>, <__main__.TableA object at 0x7ff59212fc10>, <__main__.TableA object at 0x7ff59212e500>]


In [39]:
session.delete(a)
session.commit()

2023-03-08 21:26:28,213 INFO sqlalchemy.engine.Engine DELETE FROM "T_A" WHERE "T_A".pk = ?
2023-03-08 21:26:28,214 INFO sqlalchemy.engine.Engine [generated in 0.00097s] (1,)
2023-03-08 21:26:28,216 INFO sqlalchemy.engine.Engine COMMIT


In [40]:
a.PK  # 메모리 상에서는 존재함. 그러면 코드에서 지워줘야 함

1

In [ ]:
# 이 과정으로 알 수 있는 것
# 완벽하게 "메모리에서 관리하는 인스턴스들"과 "데이터베이스에서 관리하는 데이터"의 싱크 맞추는 것이 중요함.
# 싱크를 계속 맞춰주고 객체가 살았는지 죽었는지 염두해야 함. 중간중간 커밋을 해줘야 함.
# 그렇기 때문에 session.dirty 와 is_modified()가 중요

In [41]:
session.dirty
# 세션에서 관리하고 있는 DB와 매핑되어 있는 인스턴스 사이의 변화가 있는지를 알려주는 값
# a를 DB에서 지워서 세션에서 관리하지 않음. -> 따라서 아래처럼 빈 값이 나옴
# 정리하면, Database 와 mapped 인스턴스 사이의 변화가 있는지를 살펴봄

IdentitySet([])

In [44]:
session.is_modified(a) # instance가 수정이 됐는지를 보는 것

True

In [43]:
a.NAME = 'dd'

In [45]:
session.is_modified(a)
# 객체(instance)가 수정됐는지 특히 객체는 in memory 로서, 클래스의 인스턴스임
# 이것은 실제 물리적인 DB와 상관 없음
#! 인스턴스는 메모리에 남아있기 떄문에 값이 변화됐다고 나오는 것.

True

In [46]:
session.dirty
# 지금은 왜 안나올까? -> 
# 세션은 인스턴스랑 데이터를 매핑해주는 것을 계속 관리.
# 그러나 위에서 delete로 데이터를 날렸기 때문에 값이 없음

IdentitySet([])

In [47]:
session.commit()
# 반영할 값이 없음. 위에서 delete 했기 때문.

In [48]:
type(inst)

list

In [49]:
len(inst)

4

In [50]:
inst.pop(0), len(inst) 
# 내가 인스턴스 관리하는 목록에서 지워버린 것. 즉, a라는 객체를 메모리에서도 삭제한 것.

(<__main__.TableA at 0x7ff57838b4f0>, 3)

In [ ]:
try: 
    session.begin()        # DB에서 Transaction Begin 과 같음
    session 작업들 ...
    session.commit()
except:
    session.rollback()
    
# 즉 이런식으로 세션은 관리해줌.
# 세션메이커로 엔진을 바인딩해서 우리가 이용하고 있는 것.

In [ ]:
# commit
# commit 하기 전에는 다 pending 상태

In [ ]:
# filter는 where 절이랑 같음
# query는 select와 같음
session.query(TableA).where(TableA.PK == 1).one().NAME

In [51]:
session.close()
engine.dispose()
# 이래도 인스턴스는 살아있음

In [52]:
base.metadata.clear() # 이래야 인스턴스 날아감

In [53]:
base.metadata.tables

FacadeDict({})

In [58]:
# 아래를 해주면 base에 등록된 클래스도 사라짐
base.registry.dispose()

##### **실습** 

Playlist(ORM버전)

In [60]:
# 참조 등에는 클래스 이름이 아니라, 설정한 테이블, 컬럼의 이름으로!!!!!
# => 그냥 통일시키는 게 제일 예쁠 듯

class Artist(base):
    __tablename__ = 'artist'
    
    PK = Column('pk', Integer, primary_key=True)
    NAME = Column('name', Text)
    
class Album(base):
    __tablename__ = 'album'
    
    PK = Column('pk', Integer, primary_key=True)
    NAME = Column('name', Text)
    FK = Column('fk', Integer, ForeignKey(Artist.PK))
    
class Genre(base):
    __tablename__ = 'genre'
    
    PK = Column('pk', Integer, primary_key=True)
    NAME = Column('name', Text)
    
    
class Track(base):
    __tablename__ = 'track'
    
    PK = Column('pk', Integer, primary_key=True)
    NAME = Column('name', Text)
    LENGTH = Column('length', Integer)
    RATING = Column('rating', Integer)
    COUNT = Column('count', Integer)
    FK1 = Column('fk1', Integer, ForeignKey(Album.PK))
    FK2 =  Column('fk2', Integer, ForeignKey(Genre.PK))
    

In [61]:
Album.__table__

Table('album', MetaData(), Column('pk', Integer(), table=<album>, primary_key=True, nullable=False), Column('name', Text(), table=<album>), Column('fk', Integer(), ForeignKey('artist.pk'), table=<album>), schema=None)

In [62]:
# 여기서는 클래스의 프로퍼티를 사용하는 것이기 때문에 Artist
Artist.PK

In [63]:
engine = create_engine('sqlite:///playlist_orm.db', echo=True)

In [64]:
base.metadata.create_all(engine)

2023-03-08 21:37:22,431 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 21:37:22,431 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("artist")
2023-03-08 21:37:22,432 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 21:37:22,433 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("album")
2023-03-08 21:37:22,433 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 21:37:22,434 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("genre")
2023-03-08 21:37:22,434 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 21:37:22,435 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("track")
2023-03-08 21:37:22,435 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 21:37:22,437 INFO sqlalchemy.engine.Engine COMMIT


In [65]:
Session = sessionmaker(engine)
session = Session()  # 우리가 작업할 인스턴스

In [66]:
artist1 = Artist(NAME='가수11')
artist2 = Artist(NAME='가수12')

session.add_all([artist1, artist2])
session.commit()

2023-03-08 21:37:30,044 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 21:37:30,046 INFO sqlalchemy.engine.Engine INSERT INTO artist (name) VALUES (?), (?) RETURNING pk
2023-03-08 21:37:30,047 INFO sqlalchemy.engine.Engine [generated in 0.00007s (insertmanyvalues)] ('가수11', '가수12')
2023-03-08 21:37:30,048 INFO sqlalchemy.engine.Engine COMMIT


In [67]:
# 내가 또 추가
artist1 = Artist(NAME='전영호')  # 이건 하나의 로우를 뜻함.
artist2 = Artist(NAME='마크툽')

session.add_all([artist1, artist2])
session.commit()

2023-03-08 21:38:47,923 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 21:38:47,925 INFO sqlalchemy.engine.Engine INSERT INTO artist (name) VALUES (?), (?) RETURNING pk
2023-03-08 21:38:47,926 INFO sqlalchemy.engine.Engine [cached since 77.88s ago (insertmanyvalues)] ('전영호', '마크툽')
2023-03-08 21:38:47,927 INFO sqlalchemy.engine.Engine COMMIT


In [88]:
print(session.query(Artist).all()[5].NAME)

2023-03-08 21:43:02,730 INFO sqlalchemy.engine.Engine SELECT artist.pk AS artist_pk, artist.name AS artist_name 
FROM artist
2023-03-08 21:43:02,731 INFO sqlalchemy.engine.Engine [cached since 76.39s ago] ()
마크툽


In [69]:
album = [Album(NAME='전영호_1', FK=artist1.PK),
         Album(NAME='마크툽_1', FK=artist2.PK)]

session.add_all(album)
session.commit

2023-03-08 21:39:22,803 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 21:39:22,805 INFO sqlalchemy.engine.Engine SELECT artist.pk AS artist_pk, artist.name AS artist_name 
FROM artist 
WHERE artist.pk = ?
2023-03-08 21:39:22,807 INFO sqlalchemy.engine.Engine [generated in 0.00126s] (5,)
2023-03-08 21:39:22,808 INFO sqlalchemy.engine.Engine SELECT artist.pk AS artist_pk, artist.name AS artist_name 
FROM artist 
WHERE artist.pk = ?
2023-03-08 21:39:22,809 INFO sqlalchemy.engine.Engine [cached since 0.003365s ago] (6,)


<bound method Session.commit of <sqlalchemy.orm.session.Session object at 0x7ff59212cfd0>>

In [89]:
session.add_all([Genre(NAME='락'), Genre(NAME='발라드')])
session.commit()

2023-03-08 21:43:08,197 INFO sqlalchemy.engine.Engine INSERT INTO genre (name) VALUES (?), (?) RETURNING pk
2023-03-08 21:43:08,198 INFO sqlalchemy.engine.Engine [cached since 210.9s ago (insertmanyvalues)] ('락', '발라드')
2023-03-08 21:43:08,199 INFO sqlalchemy.engine.Engine COMMIT


In [91]:
album1 = session.query(Album).where(Album.FK == artist1.PK).one()
album2 = session.query(Album).where(Album.FK == artist2.PK).one()

genre1 = session.query(Genre).where(Genre.NAME == '힙합').where(Genre.PK==1).one()
genre2 = session.query(Genre).where(Genre.NAME == '포크송').where(Genre.PK==2).one()

2023-03-08 21:43:51,096 INFO sqlalchemy.engine.Engine SELECT album.pk AS album_pk, album.name AS album_name, album.fk AS album_fk 
FROM album 
WHERE album.fk = ?
2023-03-08 21:43:51,097 INFO sqlalchemy.engine.Engine [cached since 9.104s ago] (5,)
2023-03-08 21:43:51,098 INFO sqlalchemy.engine.Engine SELECT album.pk AS album_pk, album.name AS album_name, album.fk AS album_fk 
FROM album 
WHERE album.fk = ?
2023-03-08 21:43:51,103 INFO sqlalchemy.engine.Engine [cached since 9.109s ago] (6,)
2023-03-08 21:43:51,108 INFO sqlalchemy.engine.Engine SELECT genre.pk AS genre_pk, genre.name AS genre_name 
FROM genre 
WHERE genre.name = ? AND genre.pk = ?
2023-03-08 21:43:51,111 INFO sqlalchemy.engine.Engine [generated in 0.00219s] ('힙합', 1)
2023-03-08 21:43:51,113 INFO sqlalchemy.engine.Engine SELECT genre.pk AS genre_pk, genre.name AS genre_name 
FROM genre 
WHERE genre.name = ? AND genre.pk = ?
2023-03-08 21:43:51,115 INFO sqlalchemy.engine.Engine [cached since 0.006664s ago] ('포크송', 2)


In [92]:
track = [
    Track(NAME="광대", RATING=5, LENGTH=300, COUNT=0, FK1=album1.PK, FK2=genre1.PK),
    Track(NAME="발레리노", RATING=5, LENGTH=297, COUNT=0, FK1=album1.PK, FK2=genre1.PK),
    Track(NAME="희재", RATING=5, LENGTH=312, COUNT=0, FK1=album2.PK, FK2=genre2.PK),
    Track(NAME="뜨거운 안녕", RATING=5, LENGTH=333, COUNT=0, FK1=album2.PK, FK2=genre1.PK),
    Track(NAME="회상", RATING=5, LENGTH=227, COUNT=0, FK1=album1.PK, FK2=genre2.PK),
    Track(NAME="Butterfly", RATING=5, LENGTH=277, COUNT=0, FK1=album1.PK, FK2=genre1.PK),
    Track(NAME="찰나가 빛나는 순간", RATING=5, LENGTH=297, COUNT=0, FK1=album2.PK, FK2=genre2.PK)
]
session.add_all(track)
session.commit()

2023-03-08 21:45:00,893 INFO sqlalchemy.engine.Engine INSERT INTO track (name, length, rating, count, fk1, fk2) VALUES (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?) RETURNING pk
2023-03-08 21:45:00,901 INFO sqlalchemy.engine.Engine [generated in 0.00011s (insertmanyvalues)] ('광대', 300, 5, 0, 3, 1, '발레리노', 297, 5, 0, 3, 1, '희재', 312, 5, 0, 4, 2, '뜨거운 안녕', 333, 5, 0, 4, 1, '회상', 227, 5, 0, 3, 2, 'Butterfly', 277, 5, 0, 3, 1, '찰나가 빛나는 순간', 297, 5, 0, 4, 2)
2023-03-08 21:45:00,905 INFO sqlalchemy.engine.Engine COMMIT


In [98]:
session.close() # 세션 닫기
engine.dispose() # 엔진 닫기
base.metadata.clear() # table 객체 삭제 - 베이스 날리기
base.registry.dispose() # class 삭제 - 베이스에 등록된 클래스도 날리기

In [ ]:
in-memory                                         DBMS(SQLite)
           session(instance(memory) - data(DBMS)) 
            -> 관리하고 매핑해주는 애
                    
1. base를 상속받은 Class 선언 => 메모리에서 일어난 일
2. base.metadata.tables 여기에 선언했던 클래스들의 목록이 쭉쭉 나옴.
3. base.create_all()  => 실제 물리적인 DB에 테이블을 반영해줌. 원래는 이 SQL문이 있어야 하는데 여기서는 Core가 알아서 해줌
# 모든 이 과정은 SQL문을 안쓰고 편하게 하기 위해
4. session.bind = engine  "세션이 관리하는 객체"와 "엔진이 관리하는 데이터들"이 서로 매핑
5. 메모리에 작업. Class로부터 Instance 생성 => DB에 row를 하나 Insert 하는 거랑 같음
6. 이후 세션한테 알려줌. session.add(instance) => pending 상태. 아직 반영 안됨
7. session.commit() 한 순간 이제서야 INSERT 됨. 이 commit이 완성 되었으면
우리에게 남아있는 것은 인스턴스. 그런데 instance에 있는 값들이 DB 테이블에 있는 row의 값과 동일함.
그래서 instance만 알면 DB 테이블에 뭐가 들어가 있는지 알 필요가 없음.

# 이 과정을 이해해보기!!!

In [94]:
# 작업했던 애들 불러오기
engine = create_engine('sqlite:///playlist4.db', echo=True)

In [95]:
!ls

?moon.ipynb                 moon.py
03.02(오후).ipynb           orm1.db
03.03(오전).ipynb           orm2.db
03.03(오후).ipynb           playlist.db
03.06(오전).ipynb           playlist2.db
03.06(오후).ipynb           playlist4.db
03.07(오전).ipynb           playlist_orm.db
03.07(오후).ipynb           project_exercise.py
03.08(오전).ipynb           sns.db
03.08(오후).ipynb           sns2.db
__pycache__                 test.db
example1.db                 test1.db
example2.db                 강의자료
example3.db                 연습용.py


In [96]:
base.metadata.tables

FacadeDict({})

In [ ]:
# 베이스에 테이블이 없으니 여기다가 테이블을 만들어 줘야 함.
# 그것이 먼저 클래스 만들기

In [134]:
base.metadata.reflect(engine)  # DB 에 저장된 것 불러오기

2023-03-08 11:31:02,618 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 11:31:02,619 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2023-03-08 11:31:02,620 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 11:31:02,623 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_temp_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2023-03-08 11:31:02,624 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 11:31:02,625 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("album")
2023-03-08 11:31:02,627 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 11:31:02,633 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("artist")
2023-03-08 11:31:02,639 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 11:31:02,644 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("genre")
2023-03-08 11:31:02,648 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-0

In [135]:
base.metadata.tables.keys()

dict_keys(['album', 'artist', 'genre', 'track'])

In [136]:
class Album(base):
    __table__ = base.metadata.tables['album']
    
# 이미 데이터가 있으니까 클래스 선언 따로 안하고 메타데이터를 사용함
    

In [137]:
Session = sessionmaker(engine)
session = Session()

In [139]:
albumList = session.query(Album).all()

2023-03-08 11:33:14,363 INFO sqlalchemy.engine.Engine SELECT album.pk AS album_pk, album.name AS album_name, album.fk AS album_fk 
FROM album
2023-03-08 11:33:14,364 INFO sqlalchemy.engine.Engine [cached since 13.58s ago] ()


In [142]:
albumList[0].pk, albumList[0].name   # 클래스의 인스턴스
# 이런 식으로 확인해서 튜플로 확인할 수 있음

(1, '앨범1')

In [143]:
base.metadata.tables['album'].c.pk   # column 객체

Column('pk', INTEGER(), table=<album>, primary_key=True)

In [146]:
type(base.metadata.tables['album'].c.pk)   # column 객체

sqlalchemy.sql.schema.Column

In [ ]:
class Album(base):
    __table__ = base.metadata.tables['album'] 
    # 이미 데이터가 있으니까 클래스 선언 따로 안하고 메타데이터를 사용함
    
    # 여기에 프로퍼티 = 컬럼객체 생성해야 하는데,
    # 이를 위의 메타데이터로 대체할 수 있음.

In [ ]:
# 조인 절. 여기서는 ON 절이 없음.
# 왜냐하면 앞에서 ForeignKey 등록이 되어 있어서
# 어떻게 하는지 보자

In [144]:
class Artist(base):
    __table__ = base.metadata.tables['artist']    

In [145]:
session.query(Artist).all()

2023-03-08 11:37:25,377 INFO sqlalchemy.engine.Engine SELECT artist.pk AS artist_pk, artist.name AS artist_name 
FROM artist
2023-03-08 11:37:25,378 INFO sqlalchemy.engine.Engine [generated in 0.00128s] ()


In [ ]:
# 클래스들은 registry에 들어가 있음!!

In [148]:
session.query(Artist.name, Album.name).select_from(Artist)\
    .join(Album, Album.fk==Artist.pk).all()
    
# 이거는 ON 절 쓴 것. 편하게 하려면 Foriegnkey 사용하는 것
# 뒤의 relationship이 같은 결과를 더 편하게 불러올 수 있기 때문.

2023-03-08 11:38:55,852 INFO sqlalchemy.engine.Engine SELECT artist.name AS artist_name, album.name AS album_name 
FROM artist JOIN album ON album.fk = artist.pk
2023-03-08 11:38:55,854 INFO sqlalchemy.engine.Engine [generated in 0.00110s] ()


[('가수1', '앨범1'),
 ('가수2', '앨범2'),
 ('가수3', '앨범3'),
 ('가수4', '앨범4'),
 ('가수1', '싱글1'),
 ('가수2', '싱글2'),
 ('가수3', '싱글3'),
 ('가수4', '싱글4')]

In [ ]:
# ORM 을 쓰는 것의 백미가 바로 Relationship 기능
# 이것을 쓰면 위에 한 것 조차도 필요 없음

In [ ]:
# 클래스를 이용해서 '관계'를 설정할 수 있음(클래스 사이에). 클래스들 사이에 매핑됨
# 부모자식 관계를 만들어줌
# 그 때 필요한 옵션이 back_populates / backref => 어느 클래스에 접근할지를 설정
# 복수개의 관계면 uselist 쓰면 됨 => uselist가 없으면(default가 True) N개 관계.
# uselist=False 이면 딱 하나의 관계
# 클래스에 이걸 설정하면 진짜 DB를 만질 필요가 없음
# 이것들은 FK 설정 때문에 가능함
# 이렇게 접근하면 값을 쭉 찾아오기 때문에 join을 활용한 값을 찾아오는 것과 같은 효과

#### 실습

In [99]:
base.metadata.tables

FacadeDict({})

In [100]:
# 메모리에 관련된 것은 클래스, 데이터베이스에 관련된 것은 테이블

class User(base):
    __tablename__ = 'T_USER'
    
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text, nullable=False)
    
    def __repr__(self):
        return 'PK: {}, NAME: {}'.format(self.pk, self.name)
    
class Address(base):
    __tablename__ = 'T_ADDRESS'
    
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text, nullable=False)
    fk = Column('FK', Integer, ForeignKey('T_USER.PK'))  # ForeignKey(User.pk)
    
    def __repr__(self):
        return 'PK: {}, NAME: {}'.format(self.pk, self.name, self.fk)

In [101]:
len(base.metadata.tables)

2

In [102]:
engine = create_engine('sqlite:///:memory:', echo=True)

In [103]:
base.metadata.create_all(engine)
# 이게 Table을 만드는 것

2023-03-08 21:58:19,980 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 21:58:19,981 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_USER")
2023-03-08 21:58:19,981 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 21:58:19,982 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("T_USER")
2023-03-08 21:58:19,983 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 21:58:19,984 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_ADDRESS")
2023-03-08 21:58:19,984 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 21:58:19,985 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("T_ADDRESS")
2023-03-08 21:58:19,986 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 21:58:19,987 INFO sqlalchemy.engine.Engine 
CREATE TABLE "T_USER" (
	"PK" INTEGER NOT NULL, 
	"NAME" TEXT NOT NULL, 
	PRIMARY KEY ("PK")
)


2023-03-08 21:58:19,987 INFO sqlalchemy.engine.Engine [no key 0.00095s] ()
2023-03-08 21:58:19,989 INFO sqlalchemy.engine.Engine 
CREATE TABLE "T_ADDRESS" 

In [155]:
session.close() # 세션 닫기
engine.dispose() # 엔진 닫기
base.metadata.clear() # table 객체 삭제 - 베이스 날리기
base.registry.dispose() # class 삭제 - 베이스에 등록된 클래스도 날리기

In [104]:
User(name='아무개')

PK: None, NAME: 아무개

In [105]:
Session = sessionmaker(engine)
session = Session()

In [106]:
session.add(User(name='아무개'))
session.commit()

2023-03-08 21:59:12,289 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 21:59:12,294 INFO sqlalchemy.engine.Engine INSERT INTO "T_USER" ("NAME") VALUES (?)
2023-03-08 21:59:12,297 INFO sqlalchemy.engine.Engine [generated in 0.00317s] ('아무개',)
2023-03-08 21:59:12,300 INFO sqlalchemy.engine.Engine COMMIT


In [107]:
userA = session.query(User).one()  # 인스턴스

2023-03-08 21:59:16,229 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 21:59:16,231 INFO sqlalchemy.engine.Engine SELECT "T_USER"."PK" AS "T_USER_PK", "T_USER"."NAME" AS "T_USER_NAME" 
FROM "T_USER"
2023-03-08 21:59:16,232 INFO sqlalchemy.engine.Engine [generated in 0.00118s] ()


In [108]:
userA
# 인스턴스를 보면 데이터베이스를 안봐도 되는 사례

PK: 1, NAME: 아무개

In [109]:
userB = User(name='아무개2')  # 인스턴스를 활용하는 방법 - 데이터베이스랑 상관 없음

In [110]:
session.add(userB)  # 여기서 데이터베이스에 들어간 것
session.commit()

2023-03-08 21:59:29,990 INFO sqlalchemy.engine.Engine INSERT INTO "T_USER" ("NAME") VALUES (?)
2023-03-08 21:59:29,991 INFO sqlalchemy.engine.Engine [cached since 17.7s ago] ('아무개2',)
2023-03-08 21:59:29,992 INFO sqlalchemy.engine.Engine COMMIT


In [111]:
session.add_all([Address(name='주소1', fk=userA.pk),
                 Address(name='주소2', fk=userB.pk)])
session.commit()

2023-03-08 21:59:37,180 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 21:59:37,182 INFO sqlalchemy.engine.Engine SELECT "T_USER"."PK" AS "T_USER_PK", "T_USER"."NAME" AS "T_USER_NAME" 
FROM "T_USER" 
WHERE "T_USER"."PK" = ?
2023-03-08 21:59:37,183 INFO sqlalchemy.engine.Engine [generated in 0.00074s] (1,)
2023-03-08 21:59:37,184 INFO sqlalchemy.engine.Engine SELECT "T_USER"."PK" AS "T_USER_PK", "T_USER"."NAME" AS "T_USER_NAME" 
FROM "T_USER" 
WHERE "T_USER"."PK" = ?
2023-03-08 21:59:37,185 INFO sqlalchemy.engine.Engine [cached since 0.002529s ago] (2,)
2023-03-08 21:59:37,186 INFO sqlalchemy.engine.Engine INSERT INTO "T_ADDRESS" ("NAME", "FK") VALUES (?, ?), (?, ?) RETURNING "PK"
2023-03-08 21:59:37,187 INFO sqlalchemy.engine.Engine [generated in 0.00008s (insertmanyvalues)] ('주소1', 1, '주소2', 2)
2023-03-08 21:59:37,188 INFO sqlalchemy.engine.Engine COMMIT


In [112]:
session.query(Address).all()[0]

2023-03-08 21:59:44,731 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 21:59:44,733 INFO sqlalchemy.engine.Engine SELECT "T_ADDRESS"."PK" AS "T_ADDRESS_PK", "T_ADDRESS"."NAME" AS "T_ADDRESS_NAME", "T_ADDRESS"."FK" AS "T_ADDRESS_FK" 
FROM "T_ADDRESS"
2023-03-08 21:59:44,734 INFO sqlalchemy.engine.Engine [generated in 0.00093s] ()


PK: 1, NAME: 주소1

In [113]:
session.query(Address).all()[1]

2023-03-08 21:59:46,285 INFO sqlalchemy.engine.Engine SELECT "T_ADDRESS"."PK" AS "T_ADDRESS_PK", "T_ADDRESS"."NAME" AS "T_ADDRESS_NAME", "T_ADDRESS"."FK" AS "T_ADDRESS_FK" 
FROM "T_ADDRESS"
2023-03-08 21:59:46,286 INFO sqlalchemy.engine.Engine [cached since 1.553s ago] ()


PK: 2, NAME: 주소2

In [114]:
type(session.query(Address).all()[0])

# 우리가 베이스에 등록시켜놓은 Address

2023-03-08 21:59:48,627 INFO sqlalchemy.engine.Engine SELECT "T_ADDRESS"."PK" AS "T_ADDRESS_PK", "T_ADDRESS"."NAME" AS "T_ADDRESS_NAME", "T_ADDRESS"."FK" AS "T_ADDRESS_FK" 
FROM "T_ADDRESS"
2023-03-08 21:59:48,628 INFO sqlalchemy.engine.Engine [cached since 3.896s ago] ()


__main__.Address

#### **Relationship**

In [115]:
base.registry.dispose() # base에 등록된 클래스 다 날리기
base.metadata.clear()

In [116]:
from sqlalchemy.orm import relationship

In [117]:
# 메모리에 관련된 작업은 클래스, 데이터베이스에 관련된 작업은 테이블
# 클래스 들간의 매핑이 relationship

class User(base):
    __tablename__ = 'T_USER'
    
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text, nullable=False)
    addresses = relationship('Address', back_populates='user', uselist=True) # 1
    
    def __repr__(self):
        return 'PK: {}, NAME: {}'.format(self.pk, self.name)
    
class Address(base):
    __tablename__ = 'T_ADDRESS'
    
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text, nullable=False)
    fk = Column('FK', Integer, ForeignKey('T_USER.PK'))  # ForeignKey(User.pk)
    user = relationship('User', back_populates='addresses', uselist=False) # N
    
    def __repr__(self):
        return 'PK: {}, NAME: {}, FK: {}'.format(self.pk, self.name, self.fk)

In [118]:
session.query(User).all()

2023-03-08 22:02:10,895 INFO sqlalchemy.engine.Engine SELECT "T_USER"."PK" AS "T_USER_PK", "T_USER"."NAME" AS "T_USER_NAME" 
FROM "T_USER"
2023-03-08 22:02:10,896 INFO sqlalchemy.engine.Engine [generated in 0.00103s] ()


[PK: 1, NAME: 아무개, PK: 2, NAME: 아무개2]

In [119]:
session.query(User).all()[0]

2023-03-08 22:02:13,379 INFO sqlalchemy.engine.Engine SELECT "T_USER"."PK" AS "T_USER_PK", "T_USER"."NAME" AS "T_USER_NAME" 
FROM "T_USER"
2023-03-08 22:02:13,384 INFO sqlalchemy.engine.Engine [cached since 2.489s ago] ()


PK: 1, NAME: 아무개

In [120]:
type(session.query(User).all()[0])

2023-03-08 22:02:16,037 INFO sqlalchemy.engine.Engine SELECT "T_USER"."PK" AS "T_USER_PK", "T_USER"."NAME" AS "T_USER_NAME" 
FROM "T_USER"
2023-03-08 22:02:16,038 INFO sqlalchemy.engine.Engine [cached since 5.143s ago] ()


__main__.User

In [121]:
session.query(User).all()[0].addresses[0]

2023-03-08 22:02:28,596 INFO sqlalchemy.engine.Engine SELECT "T_USER"."PK" AS "T_USER_PK", "T_USER"."NAME" AS "T_USER_NAME" 
FROM "T_USER"
2023-03-08 22:02:28,600 INFO sqlalchemy.engine.Engine [cached since 17.7s ago] ()
2023-03-08 22:02:28,603 INFO sqlalchemy.engine.Engine SELECT "T_ADDRESS"."PK" AS "T_ADDRESS_PK", "T_ADDRESS"."NAME" AS "T_ADDRESS_NAME", "T_ADDRESS"."FK" AS "T_ADDRESS_FK" 
FROM "T_ADDRESS" 
WHERE ? = "T_ADDRESS"."FK"
2023-03-08 22:02:28,608 INFO sqlalchemy.engine.Engine [generated in 0.00478s] (1,)


PK: 1, NAME: 주소1, FK: 1

In [122]:
type(session.query(User).all()[0].addresses[0])

2023-03-08 22:11:42,267 INFO sqlalchemy.engine.Engine SELECT "T_USER"."PK" AS "T_USER_PK", "T_USER"."NAME" AS "T_USER_NAME" 
FROM "T_USER"
2023-03-08 22:11:42,268 INFO sqlalchemy.engine.Engine [cached since 223.1s ago] ()


__main__.Address

In [123]:
userA = session.query(User).all()[0]

2023-03-08 22:11:43,913 INFO sqlalchemy.engine.Engine SELECT "T_USER"."PK" AS "T_USER_PK", "T_USER"."NAME" AS "T_USER_NAME" 
FROM "T_USER"
2023-03-08 22:11:43,915 INFO sqlalchemy.engine.Engine [cached since 224.7s ago] ()


In [124]:
userA.addresses[0]

PK: 1, NAME: 주소1, FK: 1

In [125]:
userA.addresses[0] is session.query(Address).all()[0]

2023-03-08 22:11:51,019 INFO sqlalchemy.engine.Engine SELECT "T_ADDRESS"."PK" AS "T_ADDRESS_PK", "T_ADDRESS"."NAME" AS "T_ADDRESS_NAME", "T_ADDRESS"."FK" AS "T_ADDRESS_FK" 
FROM "T_ADDRESS"
2023-03-08 22:11:51,021 INFO sqlalchemy.engine.Engine [generated in 0.00149s] ()


True

In [186]:
# 클래스만 매핑해줬더니, 객체의 프로퍼티를 통해서 다음 객체를 찾아갈 수 있음

In [126]:
userA.addresses[0].user is userA
# 엄청나게 편해진 방법

True

In [226]:
session.close() # 세션 닫기
engine.dispose() # 엔진 닫기
base.metadata.clear() # table 객체 삭제 - 베이스 날리기
base.registry.dispose() # class 삭제 - 베이스에 등록된 클래스도 날리기

In [169]:
# 예제는 reflect로 불러오고 클래스에 __table__ 만 하고 관계만 설정하고 __repr__ 사용

#### **실습**

In [130]:
!ls

?moon.ipynb                 moon.py
03.02(오후).ipynb           orm1.db
03.03(오전).ipynb           orm2.db
03.03(오후).ipynb           playlist.db
03.06(오전).ipynb           playlist2.db
03.06(오후).ipynb           playlist4.db
03.07(오전).ipynb           playlist_orm.db
03.07(오후).ipynb           project_exercise.py
03.08(오전).ipynb           sns.db
03.08(오후).ipynb           sns2.db
__pycache__                 test.db
example1.db                 test1.db
example2.db                 강의자료
example3.db                 연습용.py


In [227]:
engine = create_engine('sqlite:///playlist2.db', echo=True)

In [228]:
base.metadata.tables

FacadeDict({})

In [ ]:
base.metadata.reflect(engine)

2023-03-08 23:16:18,707 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-08 23:16:18,708 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2023-03-08 23:16:18,709 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 23:16:18,710 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_temp_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2023-03-08 23:16:18,710 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 23:16:18,711 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("album")
2023-03-08 23:16:18,711 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 23:16:18,712 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("artist")
2023-03-08 23:16:18,712 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-08 23:16:18,713 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("genre")
2023-03-08 23:16:18,714 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-0

In [251]:

class Album(base):
    __table__ = base.metadata.tables['album']
    
    fk = Column.append_foreign_key(Album.fk, Artist.pk)  #?
    artist = relationship('Artist', back_populates='albumList', uselist=False)
    trackList = relationship('Track', back_populates='album')

class Artist(base):
    __table__ = base.metadata.tables['artist']
    
    albumList = relationship('Album', back_populates='artist')
    
class Genre(base):
    __table__ = base.metadata.tables['genre']

    trackList = relationship('Track', back_populates='genre')
    
class Track(base):
    __table__ = base.metadata.tables['track']
    
    album = relationship('Album', back_populates='trackList', uselist = False)
    genre = relationship('Genre', back_populates='trackList', uselist = False)

TypeError: Column._set_parent() missing 2 required keyword-only arguments: 'all_names' and 'allow_replacements'

In [244]:
Album_2 = Album()


NoForeignKeysError: Could not determine join condition between parent/child tables on relationship Album.artist - there are no foreign keys linking these tables.  Ensure that referencing columns are associated with a ForeignKey or ForeignKeyConstraint, or specify a 'primaryjoin' expression.

In [230]:
base.metadata.tables.keys()

dict_keys(['album', 'artist', 'genre', 'track'])

In [210]:
print(base.metadata.tables['artist'].c)

ReadOnlyColumnCollection(artist.pk, artist.name)


In [170]:
class Artist(base):
    __tablename__ = 'artist'
    
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    
    albumList = relationship("Album", back_populates="artist", uselist=True)
    
    def __repr__(self):
        return 'PK: {}, NAME: {}'.format(self.pk, self.name)
    
class Album(base):
    __tablename__ = 'album'
    
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    fk = Column('FK', Integer, ForeignKey(Artist.pk))  # ForeignKey(User.pk)
    
    artist = relationship("Artist", back_populates="albumList", uselist=False)
    trackList = relationship("Track", back_populates="album", uselist=True)
    
    def __repr__(self):
        return 'PK: {}, NAME: {}, FK: {}'.format(self.pk, self.name, self.fk)
    
class Genre(base):
    __tablename__ = 'genre'
    
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    
    trackList = relationship("Track", back_populates="", uselist=True)
    
    def __repr__(self):
        return 'PK: {}, NAME: {}'.format(self.pk, self.name)
    
class Track(base):
    __tablename__ = 'track'
    
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    length = Column('LENGTH', Integer)
    rating = Column('RATING', Integer)
    count = Column('COUNT', Integer, default=0)
    fk1 = Column('FK1', Integer, ForeignKey(Album.pk))
    fk2 = Column('FK2', Integer, ForeignKey(Genre.pk))
    
    album = relationship('Album', back_populates="trackList", uselist=False)
    genre = relationship('Genre', back_populates="trackList", uselist=False)
    
    def __repr__(self):
        return 'PK: {}, NAME: {}, LENGTH: {}, RATING: {}, COUNT: {},\
            FK1: {}, FK2: {}'.format(self.pk, self.name, self.length, self.rating, self.count, self.fk1, self.fk2)

In [ ]:
# 기존 db 에서 foreignKey가 선언되지 않아 오류가 났음
# 위 방식처럼 처음부터 다시 하거나,
# Column 객체에서 사용 가능한(dir, help) 함수들 중,
# append_foregin_key(self, fk) 를 사용할 수도 있음

In [178]:
Session = sessionmaker(engine)
session = Session()

In [163]:
Artists = session.query(Artist).all()
Artists[1]

2023-03-08 22:18:06,004 INFO sqlalchemy.engine.Engine SELECT artist."PK" AS "artist_PK", artist."NAME" AS "artist_NAME" 
FROM artist
2023-03-08 22:18:06,005 INFO sqlalchemy.engine.Engine [cached since 31.07s ago] ()


PK: 2, NAME: 가수2

In [164]:
print(Artists[1].albumList)
# artist table의 pk == 2 를 fk 로 받는 album의 데이터들을 가져옴

2023-03-08 22:18:17,538 INFO sqlalchemy.engine.Engine SELECT album."PK" AS "album_PK", album."NAME" AS "album_NAME", album."FK" AS "album_FK" 
FROM album 
WHERE ? = album."FK"
2023-03-08 22:18:17,539 INFO sqlalchemy.engine.Engine [generated in 0.00115s] (2,)
[PK: 2, NAME: 앨범2, FK: 2, PK: 6, NAME: 싱글2, FK: 2]


In [166]:
Albums = session.query(Album).all()
Albums[0]

2023-03-08 22:20:15,207 INFO sqlalchemy.engine.Engine SELECT album."PK" AS "album_PK", album."NAME" AS "album_NAME", album."FK" AS "album_FK" 
FROM album
2023-03-08 22:20:15,208 INFO sqlalchemy.engine.Engine [cached since 7.738s ago] ()


PK: 1, NAME: 앨범1, FK: 1

In [167]:
Albums[0].trackList

2023-03-08 22:20:40,420 INFO sqlalchemy.engine.Engine SELECT track."PK" AS "track_PK", track."NAME" AS "track_NAME", track."LENGTH" AS "track_LENGTH", track."RATING" AS "track_RATING", track."COUNT" AS "track_COUNT", track."FK1" AS "track_FK1", track."FK2" AS "track_FK2" 
FROM track 
WHERE ? = track."FK1"
2023-03-08 22:20:40,422 INFO sqlalchemy.engine.Engine [generated in 0.00147s] (1,)


[PK: 2, NAME: 노래2, LENGTH: 0, RATING: 0, COUNT: 0,            FK1: 1, FK2: 2,
 PK: 6, NAME: 노래6, LENGTH: 0, RATING: 0, COUNT: 0,            FK1: 1, FK2: 1]

##### 내일 할 것

In [193]:
[0-9]{2,3}.[1-9]{4}[.]

SyntaxError: invalid syntax (4054761260.py, line 1)

In [ ]:
# 정규식을 배울 것